In [1]:
import argparse
import os

import tensorflow as tf
import numpy as np
import librosa


from audio_utils import *
from train import trainedShallowModel

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
### Constants:

#Number of frequency bins in spectrogram
N_Bins = 1025

#Number of timesteps in spectrogram
N_Timesteps = 130

#Number of training examples
N_Examples = 6705

#Number of instruments to classify
N_Instruments = 11

In [3]:
# Dictionary to map from example index to the name of the file
exampleIndexToFileName = {}

In [4]:
##### Set these variables to control reading in raw data from scratch and using preshuffled data
read_raw_data = False
use_preshuffled = True

In [5]:
if read_raw_data:
    print("Set to read raw mp3 data and generate X and Y arrays from scratch.")
else:
    print("Set to load X and Y arrays from saved files")
    
print("")

if use_preshuffled:
    print("Set to load preshuffled data to save time")
else:
    print("Set to randomly shuffle X and Y, with different shuffle every time")
    
print("\nChange the value of 'read_raw_data' or 'use_preshuffled' if this is not desired")




Set to load X and Y arrays from saved files

Set to load preshuffled data to save time

Change the value of 'read_raw_data' or 'use_preshuffled' if this is not desired


In [6]:
def getConversions():
    instrumentNames = loadList("IRMAS/instrumentNames.txt")
    instrumentAbbrevs = loadList("IRMAS/instrumentAbbrevs.txt")

    print("instrument names read from 'IRMAS/instrumentNames.txt'")
    print("instrument abbreviations read from 'IRMAS/instrumentAbbrevs.txt'")


    assert len(instrumentAbbrevs) == len(instrumentNames)

    abbrevToFullName = {}
    fullNameToAbbrev = {}

    indexToAbbrev = {}
    abbrevToIndex = {}
    for i in range(len(instrumentNames)):
        name = instrumentNames[i]
        abbrev = instrumentAbbrevs[i]

        abbrevToFullName[abbrev] = name
        fullNameToAbbrev[name] = abbrev

        indexToAbbrev[i] = abbrev
        abbrevToIndex[abbrev] = i

    conversions = (abbrevToFullName, fullNameToAbbrev, indexToAbbrev, abbrevToIndex)

    return conversions, instrumentNames, instrumentAbbrevs

In [7]:
conversions, instrumentNames, instrumentAbbrevs = getConversions()

abbrevToFullName, fullNameToAbbrev, indexToAbbrev, abbrevToIndex = conversions

analysisTools = {}
analysisTools["abbrevToFullName"] = abbrevToFullName
analysisTools["fullNameToAbbrev"] = fullNameToAbbrev
analysisTools["indexToAbbrev"] = indexToAbbrev
analysisTools["abbrevToIndex"] = abbrevToIndex
analysisTools["exampleIndexToFileName"] = exampleIndexToFileName

print("\ninstrument names = ", instrumentNames)
print("instrument abbreviations = ", instrumentAbbrevs)

instrument names read from 'IRMAS/instrumentNames.txt'
instrument abbreviations read from 'IRMAS/instrumentAbbrevs.txt'

instrument names =  ['cello', 'clarinet', 'flute', 'acoustic guitar', 'electric guitar', 'organ', 'piano', 'saxophone', 'trumpet', 'violin', 'and human singing voice']
instrument abbreviations =  ['cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi']


In [8]:
def readXAndY(instrumentAbbrevs, abbrevToIndex):
    
    X = np.zeros((N_Examples, 1, N_Timesteps, N_Bins))
    Y = np.zeros((N_Examples, N_Instruments))

    example_index = 0
    for abbrev in instrumentAbbrevs:
        print("abbrev = ", abbrev)

        instrumentIndex = abbrevToIndex[abbrev]
        y_vec = np.zeros((N_Instruments))
        y_vec[instrumentIndex] = 1


        for filename in os.listdir("IRMAS/TrainingData/" + abbrev):
            if filename.endswith(".wav"):

                Y[example_index, :] = y_vec

                completePath = "IRMAS/TrainingData/" + abbrev + "/" + filename
                spectrogram, sampleRate = read_audio_spectrum(completePath)

                curr_N_bins, curr_N_timesteps = spectrogram.shape

                assert curr_N_bins == N_Bins and curr_N_timesteps == N_Timesteps


                reshaped = np.reshape(spectrogram.T, (1,N_Timesteps, N_Bins))
                X[example_index, :, :, :] = reshaped

                example_index += 1
                print("example_index = ", example_index)
                exampleIndexToFileName[example_index] = completePath

            else:
                print("wrong file type, not wav")

    return X,Y

In [9]:
def initializeShuffledXAndY(read_raw_data, use_preshuffled):
    if read_raw_data:
        print("reading in raw data")
        X,Y = readXAndY(instrumentAbbrevs, abbrevToIndex)
        print("shape of X = ", X.shape)
        print("shape of Y = ", Y.shape)

        np.save("IRMAS/savedY", Y)
        np.save("IRMAS/savedX", X)


        shuffledX, shuffledY = shuffle(X,Y)
        np.save("IRMAS/savedXShuffled", shuffledX)
        np.save("IRMAS/savedYShuffled", shuffledY)

        print("saved X and Y in 'IRMAS/savedX' and 'IRMAS/savedY'")
        print("saved shuffled X and Y in 'IRMAS/savedXShuffled' and 'IRMAS/savedYShuffled'")

    else:
        print("using saved preprocessed data")

    #shape of X =  (6705, 1, 130, 1025)
    #shape of Y =  (6705, 11)

    x_file = "IRMAS/savedX.npy"
    y_file = "IRMAS/savedY.npy"

    x_shuffled_file = "IRMAS/savedXShuffled.npy"
    y_shuffled_file = "IRMAS/savedYShuffled.npy"

    X,Y = None, None

    if use_preshuffled:
        print("using preshuffled data")
        X,Y = loadSavedNumpyArrays(filenames = (x_shuffled_file, y_shuffled_file))
    else:
        print("Shuffling data")
        X,Y = loadSavedNumpyArrays(filenames = (x_file, y_file))
        X,Y = shuffle(X,Y)
        
        np.save("IRMAS/savedXShuffled", X)
        np.save("IRMAS/savedYShuffled", Y)

    #note: shuffle data and save into shuffled file

    return X,Y

In [10]:
X,Y = initializeShuffledXAndY(read_raw_data, use_preshuffled)

using saved preprocessed data
using preshuffled data


In [11]:
#Verify that data is shuffled, first 5 examples should have random labels
print("Y[0] = ", Y[0,:])
print("Y[1] = ", Y[1,:])
print("Y[2] = ", Y[2,:])
print("Y[3] = ", Y[3,:])
print("Y[4] = ", Y[4,:])

Y[0] =  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Y[1] =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Y[2] =  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
Y[3] =  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Y[4] =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
model = trainedShallowModel(X,Y, analysisTools)

fc name =  fc/BiasAdd:0


In [ ]:
Weights = model.trainWithMinibatch()

#np.save("IRMAS/Weights", Weights)
#print("saved final weights array in 'IRMAS/Weights'")

In [13]:
meta_file = 'savedModels/trainedShallowModel/model.meta'
checkpoint_directory = 'savedModels/trainedShallowModel/'

meta_file = 'savedModels/longerTrainedShallow/model.meta'
checkpoint_directory = 'savedModels/longerTrainedShallow/'

with tf.Session() as session:
    
    #init = tf.global_variables_initializer()
    #session.run(init)
    
    saver = tf.train.import_meta_graph(meta_file)
    saver.restore(session, tf.train.latest_checkpoint(checkpoint_directory))
    
    

    model.trainTestEval(session)

INFO:tensorflow:Restoring parameters from savedModels/longerTrainedShallow/model


FailedPreconditionError: Attempting to use uninitialized value W
	 [[Node: W/read = Identity[T=DT_FLOAT, _class=["loc:@W"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](W)]]

Caused by op 'W/read', defined at:
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/tornado/ioloop.py", l